# Data Cleaning for Dry Bean Dataset

This notebook cleans the `Dry_Bean_Dataset.xlsx` using custom functions and methodology from `clean_and_viz.ipynb`.

In [ ]:
import pandas as pd
import numpy as np
import os
import random as rand
import string
from difflib import get_close_matches

# Ensure openpyxl is installed for reading Excel files
# !pip install openpyxl

## Define Custom Functions

In [ ]:
# Create a function to find outliers using IQR
def find_outliers_IQR(df):
    #this function takes a df or only a column and calculates outliers for each value based on IQR
    
    q1 = df.quantile(0.25) # variable for the first quartile
    
    q3 = df.quantile(0.75) #variable for the third quartile
    
    IQR = q3 - q1 #calculate the IQR
    
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))] #calculate outliers for every value and put them in a series
    
    return outliers


# Range function
def range_column(df):
    max = df.max()
    min = df.min()
    range = max-min
    return range



# (NA/Null) Value Percentage Calculator
def naValues(df):
    
    for column in df:
        
        total_values = len(df.index) # number of all values in a column
        
        total_garb = df[column].isna().sum() # number off all NA/Null values (apparently null and na are same in pandas, df.isnull() == df.isna())
     
        garb_perc = (total_garb * 100) // total_values
        
        print(column, " has total of ", total_garb, " NA/Null values")
        print("NA/Null percentage of ", column," is ", garb_perc, "% \n")
            

#A custom function to replace special characters with corresponding letters
def replaceSpecialChars(text):
    # Define character replacements
    char_replacements = {
        '@': 'a',
        '3': 'e',
        '1': 'i',
        '0': 'o',
        '!': 'i',
        '#': 'h',
        '$': 's',
        '5': 's',
        '7': 't',
        '9': 'g'
    }
    # If the input is not a string, return it as-is
    if not isinstance(text, str):
        return text
    # remove empty spaces
    text = text.strip()
    # Replace each special character in the text
    for char, replacement in char_replacements.items():
        text = text.replace(char, replacement)
    
    return text
        

# A function that manipulates strings in columns.
def str_basicclean(df,column ='',style = 'cap',sp_char = string.whitespace, sp_replace = False): # capitalizes strings and strips whitespaces by default, it can strip anything if provided as argument.
    try:
        if df[column].dtype == 'object':
    
            if type(sp_char) == str:
                try:
                    df[column] = df[column].str.strip()
                    df[column] = df[column].str.strip(sp_char)
                except:
                    print('ERROR: Strip function did not work properly')
            else:
                df[column] = df[column].str.strip()
            
            if style == 'up':
                df[column] = df[column].str.upper()
            elif style == 'low':
                df[column] = df[column].str.lower()
            elif style == 'cap':
                df[column] = df[column].str.capitalize()
        
            if sp_replace == True:
                df[column] = df[column].apply(replaceSpecialChars)
    except:
        print(f'ERROR: Data Type of the {column} is not object.')
        return 0
    return df[column]



#A function to list all the unique values
def printUniqueValues(df, columnKey, sort_function=None):
    """
    Print unique values of a column from a df, with optional sorting.

    Parameters:
    df (pd.df): The df containing the column.
    columnKey (str): The key of the column whose unique values are to be printed.
    sort_function (callable, optional): A function to sort the unique values. Defaults to None.
    """
    unique_values = df[columnKey].unique().tolist()
    
    # Apply sorting if a sort function is provided
    if sort_function:
        unique_values = sorted(unique_values, key=sort_function)
    
    print(unique_values)

#A custom function to fill the empty or nan values with a selected mod
def num_filler(df, column = '', mode = 'median'):
    if df[column].dtype == 'object':
        print(f'ERROR: The data-type of column {column} is not numerical!')
    else:
        match mode:
            case 'median':
                df_median = df[column].median() 
                df[column] = df[column].fillna(df_median)
                return df[column]
            case 'mean':
                df_mean = df[column].mean()
                df[column] = df[column].fillna(df_mean)
                return df[column]
            case 'mod':
                df_mod = df[column].mod()
                df[column] = df[column].fillna(df_mod)
                return df[column]
            case 'prob':
                value_counts = df[column].value_counts()
                probabilities = value_counts / value_counts.sum()
                
                #Randomly generate values for NaN based on the probabilities
                random_values = np.random.choice(value_counts.index, size=df[column].isna().sum(), p=probabilities)
                
                #Fill the empty values with the randomly generated values
                df.loc[df[column].isna(), column] = random_values
                return df[column]
            case _:
                print('ERROR: mod is unvalid. Please choose one of the mods below:\n median(default), mean, mod, prob')
                return df[column]

def handle_incomplete_data(value, valid_values, default='Unknown', cutoff=0.4):
    if not isinstance(value, str):
        return default

    # Strip and attempt to find a close match
    value = value.strip()
    matches = get_close_matches(value, valid_values, n=1, cutoff=cutoff)
    if matches:
        return matches[0]
    else:
        return default

def process_column(df, column, valid_values, default='Unknown', cutoff=0.4):
    
    return df[column].apply(lambda x: handle_incomplete_data(x, valid_values, default, cutoff))


## Load the Dataset

In [ ]:
df = pd.read_excel('Dry_Bean_Dataset.xlsx')
display(df.head())
display(df.info())

## Check for Missing Values

In [ ]:
naValues(df)

## Data Cleaning Steps

In [ ]:
# 1. Clean Column Names
df.columns = df.columns.str.strip()

# 2. Identify potential numerical columns that might be object type due to dirtiness
#    We assume string columns like 'Class' are categorical, others likely numerical.
#    We'll try to convert everything to numeric, if it fails, we treat it as object/categorical.

exclude_cols = ['Class'] # Presumed target variable

for col in df.columns:
    if col not in exclude_cols:
        # Attempt to clean potential strings in numerical columns (e.g. "12..")
        if df[col].dtype == 'object':
             # Use replaceSpecialChars to fix things like '3' for 'e' if relevant, 
             # but for numbers we might just want to strip non-numeric chars or replacements.
             # The reference notebook used replaceSpecialChars for specific text columns.
             # Here we'll just try to force numeric conversion, handling errors.
             
             # Remove '...' if present as in the reference logic
             df[col] = df[col].astype(str).str.replace('...', '', regex=False)
             
             # Try to convert to numeric
             df[col] = pd.to_numeric(df[col], errors='coerce')
             
        # Fill missing values for numerical columns
        if pd.api.types.is_numeric_dtype(df[col]):
            num_filler(df, col, 'median')

# 3. Clean Categorical/Object Columns
for col in df.select_dtypes(include=['object']).columns:
    # Basic string cleaning (strip, capitalize)
    str_basicclean(df, col, style='cap', sp_replace=True)
    # Replacements for special chars were applied above if sp_replace=True

# Check info after cleaning
df.info()

## Outliers
Detect and handle outliers using IQR method.

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    # Just displaying outliers count for now, handling depends on preference (cap, remove, ignore)
    # The reference notebook function 'find_outliers_IQR' returns the outliers.
    outliers = find_outliers_IQR(df[col])
    print(f"{col}: {len(outliers)} outliers found.")
    
    # Optional: Fill or cap outliers? Reference notebook didn't explicitly show the 'fixing' part in the first 800 lines,
    # but often they are capped or analyzed. We will leave them for now or ask user, 
    # but typically in cleaning we might want to handle them. 
    # Given instructions, we stick to the provided functions.

## Save Cleaned Data

In [ ]:
output_file = 'Dry_Bean_Dataset_Cleaned.csv'
df.to_csv(output_file, index=False)
print(f"Cleaned dataset saved to {output_file}")